# 국토교통부_연립다세대 전월세 실거래자료_크롤링


- 데이터 출처: https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15058352   
- 지역코드 출처: 주민등록주소코드 변경내역(2018. 1. 22. 현재) https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardArticle.do?bbsId=BBSMSTR_000000000052&nttId=61552 내 첨부파일 jscode20180122.zip   
   
##### *주의: 일일 트래픽 1000회 제한*

In [55]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

## 함수 설정

In [76]:
def collect_RH_rent(ym, lawd_cd) :
    
    # 제공받은 API 
    API_KEY = 'bHg%2FpafJkgCqoGEFC3cI5IvynqkFuwK53BYhBu3bqMz7I%2BX%2BEYH8R9jV%2FCMc06dm11DO%2FksvWzs5sIUeyCjiTQ%3D%3D'
    
    # URL
    url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcRHRent'
 
    # 요청메시지
    url = url + '?&LAWD_CD=' + lawd_cd + '&DEAL_YMD=' + ym + '&serviceKey=' + API_KEY
 
    resultXML = urlopen(url)
    result = resultXML.read()
    xmlsoup = BeautifulSoup(result, 'lxml-xml')
 
    te = xmlsoup.findAll('item')
 
    # 데이터프레임 결과물
    RH_rent = pd.DataFrame()
 
    for t in te:
        deposit = t.find('보증금액').text
        mon_rent = t.find('월세금액').text
        try:
            build_y = t.find("건축년도").text
        except:
            build_y = ""
        deal_y = t.find('년').text
        deal_m = t.find('월').text
        deal_day = t.find('일').text
        try:
            RH_nm = t.find("연립다세대").text
        except:
            RH_nm = ""
        area_exc = t.find('전용면적').text
        dong = t.find('법정동').text
        try:
            region_num = t.find("지번").text
        except:
            region_num = ""
 
        floor = t.find('층').text
        region_code = t.find('지역코드').text
 
        temp = pd.DataFrame(([[deposit, mon_rent, build_y, deal_y, deal_m, deal_day, RH_nm, area_exc, dong, region_num, floor, region_code]]),
                            columns = ['deposit', 'mon_rent', 'build_y', 'deal_y', 'deal_m', 'deal_day', 'RH_nm', 'area_exc', 'dong', 'region_num', 'floor', 'region_code'])
        RH_rent = pd.concat([RH_rent, temp])
 
    RH_rent = RH_rent.reset_index(drop = True)
 
    return RH_rent

## 수집

#### 수집 경과 ####
2020년 data => 7/16 완료    
2019년 data => 7/16 트래픽 초과 => 2019년 2월 경기도 일산동구 데이터까지 수집    
2018년 data => 7/16 트래픽 초과

### **** 2019년 2월 경기도 데이터 이후 수집 예정 ****

In [67]:
if __name__ == "__main__":

    # 지역코드 가져오기
    code = pd.read_excel("KIKcd_B.20180122.xlsx")
    code_cap = code[(code["시도명"] == "경기도")]
    code_cap = code_cap[code_cap["읍면동명"].isnull() == True]
    code_cap = code_cap[code_cap["시군구명"].isnull() == False]
    code_cap["region_code"] = code_cap["법정동코드"].astype(str).str[0:5]
    code_cap = code_cap.reset_index(drop=True)
    RH_rent = pd.DataFrame()

    # 수집 기간 설정(연월: 'yyyymm')
    ym = list(['201907'])
 
    for m in ym:
        i = 0
        for co in code_cap['region_code'] :
            temp = collect_RH_rent(m, co)
            temp['sido'] = code_cap['시도명'][i]
            temp['sigungu'] = code_cap['시군구명'][i]
            RH_rent = pd.concat([RH_rent, temp])
            i += 1
            print(co + ", " + str(len(temp)) + "개 완료")
            
        print("******** " + str(m) + " 완료 ********")

41110, 0개 완료
41111, 83개 완료
41113, 155개 완료
41115, 137개 완료
41117, 36개 완료
41130, 0개 완료
41131, 96개 완료
41133, 160개 완료
41135, 74개 완료
41150, 145개 완료
41170, 0개 완료
41171, 174개 완료
41173, 117개 완료
41190, 497개 완료
41210, 80개 완료
41220, 98개 완료
41250, 25개 완료
41270, 0개 완료
41271, 372개 완료
41273, 138개 완료
41280, 0개 완료
41281, 126개 완료
41285, 60개 완료
41287, 48개 완료
41290, 29개 완료
41310, 49개 완료
41360, 227개 완료
41370, 90개 완료
41390, 111개 완료
41410, 79개 완료
41430, 70개 완료
41450, 27개 완료
41460, 0개 완료
41461, 108개 완료
41463, 79개 완료
41465, 63개 완료
41480, 78개 완료
41500, 56개 완료
41550, 35개 완료
41570, 43개 완료
41590, 133개 완료
41610, 223개 완료
41630, 27개 완료
41650, 37개 완료
41670, 18개 완료
41800, 10개 완료
41820, 19개 완료
41830, 37개 완료
******** 201907 완료 ********


In [68]:
RH_rent_20190307 = pd.concat([RH_rent_20190307, RH_rent])

In [69]:
RH_rent_20190307

,deposit,mon_rent,build_y,deal_y,deal_m,deal_day,RH_nm,area_exc,dong,region_num,floor,region_code,sido,sigungu
0,"5,000",50,1997,2019,3,9,(17-28),47.57,신교동,17-28,3,11110,서울특별시,종로구
1,"15,000",0,1997,2019,3,16,(17-41),40.97,신교동,17-41,1,11110,서울특별시,종로구
2,"25,000",0,1996,2019,3,20,(14-5),57.9,신교동,14-5,2,11110,서울특별시,종로구
3,"3,000",100,1996,2019,3,23,(50-4),58.83,신교동,50-4,1,11110,서울특별시,종로구
4,"15,000",0,1996,2019,3,26,(29-0),46.49,신교동,29,2,11110,서울특별시,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,"13,650",0,2012,2019,7,31,(402-1),74.41,서종면 문호리,402-1,3,41830,경기도,양평군
33,"5,500",0,1998,2019,7,2,용문빌라(389),65.25,용문면 다문리,389,2,41830,경기도,양평군
34,"7,000",0,2012,2019,7,4,용문블루스위트2차(A동),47.24,용문면 다문리,481-3,2,41830,경기도,양평군
35,"17,000",0,2018,2019,7,20,개성하이빌,59.72,용문면 다문리,792-4,7,41830,경기도,양평군


In [72]:
RH_rent_20190207 = pd.concat([RH_rent_201902, RH_rent_20190307])

In [74]:
RH_rent_20190207

,deposit,mon_rent,build_y,deal_y,deal_m,deal_day,RH_nm,area_exc,dong,region_num,floor,region_code,sido,sigungu
0,"36,000",0,2010,2019,1,16,효산빌리지,57.71,청운동,52-5,2,11110,서울특별시,종로구
1,"9,000",100,2009,2019,1,25,아름드리청운빌,66.65,청운동,60,3,11110,서울특별시,종로구
2,"3,000",70,1997,2019,1,25,(17-36),50.24,신교동,17-36,3,11110,서울특별시,종로구
3,"20,000",0,1998,2019,1,29,(17-14),47.04,신교동,17-14,4,11110,서울특별시,종로구
4,"38,000",0,2017,2019,1,29,팔라디오빌,58.05,신교동,17-10,2,11110,서울특별시,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,"13,650",0,2012,2019,7,31,(402-1),74.41,서종면 문호리,402-1,3,41830,경기도,양평군
33,"5,500",0,1998,2019,7,2,용문빌라(389),65.25,용문면 다문리,389,2,41830,경기도,양평군
34,"7,000",0,2012,2019,7,4,용문블루스위트2차(A동),47.24,용문면 다문리,481-3,2,41830,경기도,양평군
35,"17,000",0,2018,2019,7,20,개성하이빌,59.72,용문면 다문리,792-4,7,41830,경기도,양평군


In [75]:
RH_rent_20190207.to_csv('RH_rent_201907.csv', index = False)

In [77]:
if __name__ == "__main__":

    # 지역코드 가져오기
    code = pd.read_excel("KIKcd_B.20180122.xlsx")
    code_cap = code[(code["시도명"]=="서울특별시") | (code["시도명"] == "경기도")]
    code_cap = code_cap[code_cap["읍면동명"].isnull() == True]
    code_cap = code_cap[code_cap["시군구명"].isnull() == False]
    code_cap["region_code"] = code_cap["법정동코드"].astype(str).str[0:5]
    code_cap = code_cap.reset_index(drop=True)
    RH_rent = pd.DataFrame()

    # 수집 기간 설정(연월: 'yyyymm')
    ym = list(['201908', '201909', '201910', '201911', '201912'])
 
    for m in ym:
        i = 0
        for co in code_cap['region_code'] :
            temp = collect_RH_rent(m, co)
            temp['sido'] = code_cap['시도명'][i]
            temp['sigungu'] = code_cap['시군구명'][i]
            RH_rent = pd.concat([RH_rent, temp])
            i += 1
            print(co + ", " + str(len(temp)) + "개 완료")
            
        print("******** " + str(m) + " 완료 ********")

11110, 96개 완료
11140, 74개 완료
11170, 227개 완료
11200, 87개 완료
11215, 417개 완료
11230, 191개 완료
11260, 288개 완료
11290, 301개 완료
11305, 133개 완료
11320, 189개 완료
11350, 114개 완료
11380, 635개 완료
11410, 235개 완료
11440, 442개 완료
11470, 382개 완료
11500, 730개 완료
11530, 265개 완료
11545, 167개 완료
11560, 180개 완료
11590, 316개 완료
11620, 345개 완료
11650, 434개 완료
11680, 396개 완료
11710, 1027개 완료
11740, 536개 완료
41110, 0개 완료
41111, 88개 완료
41113, 135개 완료
41115, 135개 완료
41117, 25개 완료
41130, 0개 완료
41131, 59개 완료
41133, 140개 완료
41135, 69개 완료
41150, 108개 완료
41170, 0개 완료
41171, 137개 완료
41173, 118개 완료
41190, 486개 완료
41210, 74개 완료
41220, 123개 완료
41250, 27개 완료
41270, 0개 완료
41271, 362개 완료
41273, 117개 완료
41280, 0개 완료
41281, 134개 완료
41285, 46개 완료
41287, 41개 완료
41290, 32개 완료
41310, 52개 완료
41360, 181개 완료
41370, 74개 완료
41390, 119개 완료
41410, 84개 완료
41430, 94개 완료
41450, 27개 완료
41460, 0개 완료
41461, 122개 완료
41463, 50개 완료
41465, 51개 완료
41480, 83개 완료
41500, 43개 완료
41550, 23개 완료
41570, 52개 완료
41590, 107개 완료
41610, 208개 완료
41630, 48개 완료
41650, 32개 완료
4

In [78]:
RH_rent

,deposit,mon_rent,build_y,deal_y,deal_m,deal_day,RH_nm,area_exc,dong,region_num,floor,region_code,sido,sigungu
0,"50,000",0,1988,2019,8,1,청운벽산빌리지,83.61,청운동,1,1,11110,서울특별시,종로구
1,"50,000",0,2002,2019,8,2,아트빌,84.96,청운동,89-5,1,11110,서울특별시,종로구
2,"50,000",0,2002,2019,8,2,아트빌,84.82,청운동,89-5,2,11110,서울특별시,종로구
3,"1,500",60,1997,2019,8,8,오케이공인,34.74,신교동,32,3,11110,서울특별시,종로구
4,"20,000",0,1996,2019,8,9,(17-3),55.5,신교동,17-3,4,11110,서울특별시,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,500,35,2002,2019,12,10,화인빌라,42.55,용문면 다문리,522-2,4,41830,경기도,양평군
26,"12,000",0,2017,2019,12,16,탑클래스A동,48.03,용문면 다문리,703-26,5,41830,경기도,양평군
27,"11,000",0,2017,2019,12,16,탑클래스A동,43.41,용문면 다문리,703-26,2,41830,경기도,양평군
28,"9,000",0,2017,2019,12,16,탑클래스A동,42.72,용문면 다문리,703-26,5,41830,경기도,양평군


In [82]:
RH_rent_2019 = pd.concat([RH_rent_201907, RH_rent])

In [80]:
RH_rent_201907 = pd.read_csv('RH_rent_201907.csv')

C:\Users\shic\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
RH_rent_2019 = pd.concat([RH_rent_201907, RH_rent])

In [83]:
RH_rent_2019.to_csv('RH_rent_2019_new.csv', index=False)

In [63]:
RH_rent_20190307 = pd.read_csv('RH_rent_20190307.csv')

In [81]:
RH_rent_201907

,deposit,mon_rent,build_y,deal_y,deal_m,deal_day,RH_nm,area_exc,dong,region_num,floor,region_code,sido,sigungu
0,"36,000",0,2010,2019,1,16,효산빌리지,57.71,청운동,52-5,2,11110,서울특별시,종로구
1,"9,000",100,2009,2019,1,25,아름드리청운빌,66.65,청운동,60,3,11110,서울특별시,종로구
2,"3,000",70,1997,2019,1,25,(17-36),50.24,신교동,17-36,3,11110,서울특별시,종로구
3,"20,000",0,1998,2019,1,29,(17-14),47.04,신교동,17-14,4,11110,서울특별시,종로구
4,"38,000",0,2017,2019,1,29,팔라디오빌,58.05,신교동,17-10,2,11110,서울특별시,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88796,"13,650",0,2012,2019,7,31,(402-1),74.41,서종면 문호리,402-1,3,41830,경기도,양평군
88797,"5,500",0,1998,2019,7,2,용문빌라(389),65.25,용문면 다문리,389,2,41830,경기도,양평군
88798,"7,000",0,2012,2019,7,4,용문블루스위트2차(A동),47.24,용문면 다문리,481-3,2,41830,경기도,양평군
88799,"17,000",0,2018,2019,7,20,개성하이빌,59.72,용문면 다문리,792-4,7,41830,경기도,양평군


In [65]:
RH_rent_20190307.drop(RH_rent_20190307.loc[(RH_rent_20190307['sido']=='경기도') & (RH_rent_20190307['deal_m'] == 7)].index, inplace=True)

In [66]:
RH_rent_20190307

,deposit,mon_rent,build_y,deal_y,deal_m,deal_day,RH_nm,area_exc,dong,region_num,floor,region_code,sido,sigungu
0,"5,000",50,1997,2019,3,9,(17-28),47.57,신교동,17-28,3,11110,서울특별시,종로구
1,"15,000",0,1997,2019,3,16,(17-41),40.97,신교동,17-41,1,11110,서울특별시,종로구
2,"25,000",0,1996,2019,3,20,(14-5),57.90,신교동,14-5,2,11110,서울특별시,종로구
3,"3,000",100,1996,2019,3,23,(50-4),58.83,신교동,50-4,1,11110,서울특별시,종로구
4,"15,000",0,1996,2019,3,26,(29-0),46.49,신교동,29,2,11110,서울특별시,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58712,"20,000",0,2015,2019,7,31,일호홈타운,39.37,천호동,231-25,3,11740,서울특별시,강동구
58713,"32,000",0,2019,2019,7,31,한강팰리스,39.79,천호동,391-37,3,11740,서울특별시,강동구
58714,"5,000",55,2019,2019,7,31,청운캐슬,25.98,천호동,575,5,11740,서울특별시,강동구
58715,"13,000",0,2018,2019,7,31,예건네스빌(167-175),14.97,천호동,167-175,9,11740,서울특별시,강동구
